In [ ]:
import pandas as pd
import csv
import os
import numpy as np
import nltk
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/THESIS/AiSecKG Models/train.csv')

# TransE

In [ ]:
train.head(20)

,DocNo,sentence,tag
0,D1,Attacks,B-ATTACK
1,D1,can,O
2,D1,damage,O
3,D1,public,B-SYSTEM
4,D1,domain,I-SYSTEM
5,NaN,NaN,NaN
6,D1,IDS,B-TOOL
7,D1,means,O
8,D1,Intrusion,B-TECHNIQUE
9,D1,Detection,I-TECHNIQUE


In [ ]:
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim



# Drop rows with missing values
data = train.dropna()

# Step 1: Create mappings for entities and relations
entities = list(set(data['DocNo'].unique()).union(set(data['tag'].unique())))
relations = list(data['sentence'].unique())

entity2id = {entity: idx for idx, entity in enumerate(entities)}
relation2id = {relation: idx for idx, relation in enumerate(relations)}

# Step 2: Convert dataset to IDs
triples = data.apply(lambda row: (entity2id[row['DocNo']],
                                  relation2id[row['sentence']],
                                  entity2id[row['tag']]), axis=1).tolist()

# TransE Model
class TransE(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        super(TransE, self).__init__()
        self.entity_embedding = nn.Embedding(num_entities, embedding_dim)
        self.relation_embedding = nn.Embedding(num_relations, embedding_dim)
        nn.init.xavier_uniform_(self.entity_embedding.weight)
        nn.init.xavier_uniform_(self.relation_embedding.weight)

    def forward(self, heads, relations, tails):
        h = self.entity_embedding(heads)
        r = self.relation_embedding(relations)
        t = self.entity_embedding(tails)
        return h + r - t

# Custom Dataset
class TripleDataset(Dataset):
    def __init__(self, triples):
        self.triples = triples

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        return torch.tensor(self.triples[idx])

# Training Setup
embedding_dim = 50
num_epochs = 50
learning_rate = 0.0001
batch_size = 32

model = TransE(len(entities), len(relations), embedding_dim)
dataset = TripleDataset(triples)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Add this function to compute accuracy
def compute_accuracy(model, triples, entity2id):
    """
    Compute accuracy by checking if the true tail is ranked first
    among all possible tails based on the model's predictions.
    """
    correct = 0
    total = len(triples)

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for head, relation, tail in triples:
            # Get embeddings for the head and relation
            h = model.entity_embedding(torch.tensor(head))
            r = model.relation_embedding(torch.tensor(relation))
            true_tail = model.entity_embedding(torch.tensor(tail))

            # Compute scores for all entities as potential tails
            scores = torch.norm(h + r - model.entity_embedding.weight, dim=1)
            predicted_tail = torch.argmin(scores)  # Predicted closest tail

            # Check if the prediction matches the true tail
            if predicted_tail.item() == tail:
                correct += 1

    accuracy = correct / total
    return accuracy

# Training Loop with Accuracy Calculation
for epoch in range(num_epochs):
    total_loss = 0
    model.train()  # Set the model to training mode

    for batch in dataloader:
        heads, relations, tails = batch[:, 0], batch[:, 1], batch[:, 2]
        predictions = model(heads, relations, tails)
        loss = criterion(predictions, torch.zeros_like(predictions))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    # Compute training accuracy
    train_accuracy = compute_accuracy(model, triples, entity2id)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.2f}, Accuracy: {train_accuracy:.2f}")

print("Training complete.")



Epoch 1/50, Loss: 8.62, Accuracy: 0.00
Epoch 2/50, Loss: 6.99, Accuracy: 0.00
Epoch 3/50, Loss: 5.67, Accuracy: 0.00
Epoch 4/50, Loss: 4.60, Accuracy: 0.00
Epoch 5/50, Loss: 3.73, Accuracy: 0.00
Epoch 6/50, Loss: 3.03, Accuracy: 0.00
Epoch 7/50, Loss: 2.46, Accuracy: 0.00
Epoch 8/50, Loss: 2.00, Accuracy: 0.00
Epoch 9/50, Loss: 1.62, Accuracy: 0.05
Epoch 10/50, Loss: 1.32, Accuracy: 0.08
Epoch 11/50, Loss: 1.08, Accuracy: 0.15
Epoch 12/50, Loss: 0.89, Accuracy: 0.21
Epoch 13/50, Loss: 0.73, Accuracy: 0.25
Epoch 14/50, Loss: 0.60, Accuracy: 0.29
Epoch 15/50, Loss: 0.50, Accuracy: 0.32
Epoch 16/50, Loss: 0.42, Accuracy: 0.36
Epoch 17/50, Loss: 0.36, Accuracy: 0.40
Epoch 18/50, Loss: 0.30, Accuracy: 0.46
Epoch 19/50, Loss: 0.26, Accuracy: 0.50
Epoch 20/50, Loss: 0.22, Accuracy: 0.55
Epoch 21/50, Loss: 0.20, Accuracy: 0.58
Epoch 22/50, Loss: 0.17, Accuracy: 0.62
Epoch 23/50, Loss: 0.15, Accuracy: 0.64
Epoch 24/50, Loss: 0.13, Accuracy: 0.67
Epoch 25/50, Loss: 0.12, Accuracy: 0.70
Epoch 26/

In [ ]:
test=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/THESIS/AiSecKG Models/test.csv')

In [9]:
test.head(20)

,DocNo,sentence,tag
0,D4,you,O
1,D4,need,O
2,D4,to,O
3,D4,use,O
4,D4,Metasploit,B-TOOL
5,D4,Framework,I-TOOL
6,D4,msf,I-TOOL
7,D4,to,O
8,D4,deploy,B-FUNCTION
9,D4,at,O


# DistMult

## Datset

In [12]:
train.head(20)

,DocNo,sentence,tag
0,D1,Attacks,B-ATTACK
1,D1,can,O
2,D1,damage,O
3,D1,public,B-SYSTEM
4,D1,domain,I-SYSTEM
5,NaN,NaN,NaN
6,D1,IDS,B-TOOL
7,D1,means,O
8,D1,Intrusion,B-TECHNIQUE
9,D1,Detection,I-TECHNIQUE


## DistMult Model train

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Step 1: Prepare the Dataset
# Assuming 'train' is a DataFrame with columns ['DocNo', 'sentence', 'tag']
# Create mappings for entities and relations
entities = list(set(train['DocNo'].unique()).union(set(train['tag'].unique())))
relations = list(train['sentence'].unique())

entity2id = {entity: idx for idx, entity in enumerate(entities)}
relation2id = {relation: idx for idx, relation in enumerate(relations)}

# Convert train dataset to triples
triples = train.apply(lambda row: (entity2id[row['DocNo']],
                                   relation2id[row['sentence']],
                                   entity2id[row['tag']]), axis=1).tolist()

# Step 2: Define DistMult Model
class DistMult(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        super(DistMult, self).__init__()
        self.entity_embedding = nn.Embedding(num_entities, embedding_dim)
        self.relation_embedding = nn.Embedding(num_relations, embedding_dim)
        nn.init.xavier_uniform_(self.entity_embedding.weight)
        nn.init.xavier_uniform_(self.relation_embedding.weight)

    def forward(self, heads, relations, tails):
        h = self.entity_embedding(heads)
        r = self.relation_embedding(relations)
        t = self.entity_embedding(tails)
        scores = torch.sum(h * r * t, dim=1)  # Bilinear score computation
        return scores

# Step 3: Define Dataset and DataLoader
class TripleDataset(Dataset):
    def __init__(self, triples):
        self.triples = triples

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        return torch.tensor(self.triples[idx])

# Step 4: Negative Sampling Function
def generate_negative_samples(triples, num_entities, num_samples=1):
    negative_triples = []
    for head, relation, tail in triples:
        for _ in range(num_samples):
            if torch.rand(1).item() < 0.5:  # Corrupt head
                corrupt_head = torch.randint(0, num_entities, (1,)).item()
                while corrupt_head == head:  # Ensure it's different
                    corrupt_head = torch.randint(0, num_entities, (1,)).item()
                negative_triples.append((corrupt_head, relation, tail))
            else:  # Corrupt tail
                corrupt_tail = torch.randint(0, num_entities, (1,)).item()
                while corrupt_tail == tail:  # Ensure it's different
                    corrupt_tail = torch.randint(0, num_entities, (1,)).item()
                negative_triples.append((head, relation, corrupt_tail))
    return negative_triples

# Step 5: Training Setup
embedding_dim = 50
num_epochs = 100
learning_rate = 0.0001
batch_size = 32
negative_samples_per_positive = 5

model = DistMult(len(entities), len(relations), embedding_dim)
dataset = TripleDataset(triples)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Step 6: Training Loop
for epoch in range(num_epochs):
    total_loss = 0
    model.train()

    for batch in dataloader:
        positive_heads, positive_relations, positive_tails = batch[:, 0], batch[:, 1], batch[:, 2]

        # Generate negative samples
        negatives = generate_negative_samples(batch.tolist(), len(entities), negative_samples_per_positive)
        negative_heads = torch.tensor([neg[0] for neg in negatives])
        negative_relations = torch.tensor([neg[1] for neg in negatives])
        negative_tails = torch.tensor([neg[2] for neg in negatives])

        # Concatenate positive and negative samples
        all_heads = torch.cat([positive_heads, negative_heads])
        all_relations = torch.cat([positive_relations, negative_relations])
        all_tails = torch.cat([positive_tails, negative_tails])

        # Create labels: 1 for positive, 0 for negative
        labels = torch.cat([torch.ones(len(positive_heads)), torch.zeros(len(negative_heads))])

        # Compute scores
        scores = model(all_heads, all_relations, all_tails)

        # Compute loss
        loss = criterion(scores, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss:.2f}")

print("Training complete.")


Epoch 1/100, Loss: 125.46
Epoch 2/100, Loss: 125.39
Epoch 3/100, Loss: 125.32
Epoch 4/100, Loss: 125.23
Epoch 5/100, Loss: 125.10
Epoch 6/100, Loss: 124.92
Epoch 7/100, Loss: 124.64
Epoch 8/100, Loss: 124.25
Epoch 9/100, Loss: 123.69
Epoch 10/100, Loss: 122.96
Epoch 11/100, Loss: 121.94
Epoch 12/100, Loss: 120.74
Epoch 13/100, Loss: 119.24
Epoch 14/100, Loss: 117.36
Epoch 15/100, Loss: 115.29
Epoch 16/100, Loss: 112.99
Epoch 17/100, Loss: 110.61
Epoch 18/100, Loss: 107.74
Epoch 19/100, Loss: 104.88
Epoch 20/100, Loss: 102.14
Epoch 21/100, Loss: 98.99
Epoch 22/100, Loss: 96.10
Epoch 23/100, Loss: 93.04
Epoch 24/100, Loss: 90.26
Epoch 25/100, Loss: 87.47
Epoch 26/100, Loss: 84.62
Epoch 27/100, Loss: 81.85
Epoch 28/100, Loss: 79.22
Epoch 29/100, Loss: 76.77
Epoch 30/100, Loss: 74.55
Epoch 31/100, Loss: 72.35
Epoch 32/100, Loss: 69.87
Epoch 33/100, Loss: 67.74
Epoch 34/100, Loss: 65.44
Epoch 35/100, Loss: 63.66
Epoch 36/100, Loss: 62.13
Epoch 37/100, Loss: 59.98
Epoch 38/100, Loss: 58.55
E

Test Accuracy: 0.00


# RotatE

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import pandas as pd

# Load your training data
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/THESIS/AiSecKG Models/train.csv')

# Step 1: Create mappings for entities and relations
entities = list(set(train['DocNo'].unique()).union(set(train['tag'].unique())))
relations = list(train['sentence'].unique())

entity2id = {entity: idx for idx, entity in enumerate(entities)}
relation2id = {relation: idx for idx, relation in enumerate(relations)}

# Step 2: Convert dataset to IDs
triples = train.apply(lambda row: (entity2id[row['DocNo']],
                                  relation2id[row['sentence']],
                                  entity2id[row['tag']]), axis=1).tolist()

# RotatE Model Definition
class RotatE(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim, margin=6.0):
        super(RotatE, self).__init__()

        self.embedding_dim = embedding_dim
        self.margin = margin

        # Entity and relation embeddings (complex-valued)
        self.entity_embedding_real = nn.Embedding(num_entities, embedding_dim)
        self.entity_embedding_imag = nn.Embedding(num_entities, embedding_dim)
        self.relation_embedding_real = nn.Embedding(num_relations, embedding_dim)
        self.relation_embedding_imag = nn.Embedding(num_relations, embedding_dim)

        # Xavier Initialization
        nn.init.xavier_uniform_(self.entity_embedding_real.weight)
        nn.init.xavier_uniform_(self.entity_embedding_imag.weight)
        nn.init.xavier_uniform_(self.relation_embedding_real.weight)
        nn.init.xavier_uniform_(self.relation_embedding_imag.weight)

    def forward(self, heads, relations, tails):
        # Get the real and imaginary parts of the embeddings
        h_real = self.entity_embedding_real(heads)
        h_imag = self.entity_embedding_imag(heads)
        t_real = self.entity_embedding_real(tails)
        t_imag = self.entity_embedding_imag(tails)
        r_real = self.relation_embedding_real(relations)
        r_imag = self.relation_embedding_imag(relations)

        # Apply the rotation operation (real part of the relation rotation)
        rotated_real = h_real + r_real
        rotated_imag = h_imag + r_imag

        # The goal is to minimize the difference between the rotated head and the tail
        score = torch.norm((rotated_real - t_real) ** 2 + (rotated_imag - t_imag) ** 2, p=2, dim=1)

        return score

    def loss(self, positive_score, negative_score):
        # Margin Ranking Loss (Ensure positive scores are less than negative)
        return torch.sum(torch.clamp(positive_score - negative_score + self.margin, min=0))

# Custom Dataset for RotatE
class TripleDataset(Dataset):
    def __init__(self, triples):
        self.triples = triples

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        return torch.tensor(self.triples[idx])

# Training Setup
embedding_dim = 100
num_epochs = 100
learning_rate = 0.001
batch_size = 32
margin = 6.0

model = RotatE(len(entities), len(relations), embedding_dim, margin)
dataset = TripleDataset(triples)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in dataloader:
        heads, relations, tails = batch[:, 0], batch[:, 1], batch[:, 2]

        # Positive score
        positive_score = model(heads, relations, tails)

        # Negative sampling: replace tails with random entities for negative samples
        negative_tails = torch.randint(0, len(entities), (batch.size(0),))

        # Negative score
        negative_score = model(heads, relations, negative_tails)

        # Compute loss
        loss = model.loss(positive_score, negative_score)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.2f}")

print("Training Complete.")


Epoch 1/100, Loss: 29844.88
Epoch 2/100, Loss: 16746.53
Epoch 3/100, Loss: 12260.25
Epoch 4/100, Loss: 10524.89
Epoch 5/100, Loss: 9564.14
Epoch 6/100, Loss: 8555.33
Epoch 7/100, Loss: 8003.09
Epoch 8/100, Loss: 7612.41
Epoch 9/100, Loss: 7251.82
Epoch 10/100, Loss: 6857.29
Epoch 11/100, Loss: 6580.04
Epoch 12/100, Loss: 6025.35
Epoch 13/100, Loss: 5670.32
Epoch 14/100, Loss: 5945.92
Epoch 15/100, Loss: 5521.25
Epoch 16/100, Loss: 5284.10
Epoch 17/100, Loss: 5186.38
Epoch 18/100, Loss: 4938.48
Epoch 19/100, Loss: 5013.00
Epoch 20/100, Loss: 4774.11
Epoch 21/100, Loss: 4697.03
Epoch 22/100, Loss: 4270.16
Epoch 23/100, Loss: 4666.99
Epoch 24/100, Loss: 4150.96
Epoch 25/100, Loss: 4040.44
Epoch 26/100, Loss: 3900.32
Epoch 27/100, Loss: 3832.76
Epoch 28/100, Loss: 3804.67
Epoch 29/100, Loss: 3804.28
Epoch 30/100, Loss: 3646.98
Epoch 31/100, Loss: 3733.18
Epoch 32/100, Loss: 3393.07
Epoch 33/100, Loss: 3402.40
Epoch 34/100, Loss: 3435.90
Epoch 35/100, Loss: 3531.25
Epoch 36/100, Loss: 3174.

# ComplEx

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader

# Load your test data
test_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/THESIS/AiSecKG Models/test.csv')

# Step 1: Prepare entities and relations
entities = list(set(test_data['DocNo'].unique()).union(set(test_data['tag'].unique())))
relations = list(test_data['sentence'].unique())

# Map entities and relations to indices
entity2id = {entity: idx for idx, entity in enumerate(entities)}
relation2id = {relation: idx for idx, relation in enumerate(relations)}

# Convert data to triples in the form of (head, relation, tail)
test_triples = test_data.apply(lambda row: (entity2id[row['DocNo']],
                                            relation2id[row['sentence']],
                                            entity2id[row['tag']]), axis=1).tolist()

# ComplEx Model
class ComplEx(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        super(ComplEx, self).__init__()
        # Real and Imaginary parts for entities and relations
        self.entity_embedding_real = nn.Embedding(num_entities, embedding_dim)
        self.entity_embedding_imag = nn.Embedding(num_entities, embedding_dim)
        self.relation_embedding_real = nn.Embedding(num_relations, embedding_dim)
        self.relation_embedding_imag = nn.Embedding(num_relations, embedding_dim)

        # Initialize weights
        nn.init.xavier_uniform_(self.entity_embedding_real.weight)
        nn.init.xavier_uniform_(self.entity_embedding_imag.weight)
        nn.init.xavier_uniform_(self.relation_embedding_real.weight)
        nn.init.xavier_uniform_(self.relation_embedding_imag.weight)

    def forward(self, heads, relations, tails):
        # Get embeddings
        h_real = self.entity_embedding_real(heads)
        h_imag = self.entity_embedding_imag(heads)
        r_real = self.relation_embedding_real(relations)
        r_imag = self.relation_embedding_imag(relations)
        t_real = self.entity_embedding_real(tails)
        t_imag = self.entity_embedding_imag(tails)

        # ComplEx scoring function (real and imaginary parts)
        score = (h_real * t_real + h_imag * t_imag) + \
                (r_real * t_real + r_imag * t_imag) - \
                (h_real * r_real + h_imag * r_imag)
        return score

# Dataset Class
class TripleDataset(Dataset):
    def __init__(self, triples):
        self.triples = triples

    def __len__(self):
        return len(self.triples)

    def __getitem__(self, idx):
        head, relation, tail = self.triples[idx]
        return torch.tensor(head), torch.tensor(relation), torch.tensor(tail)

# Step 2: Initialize dataset and dataloader
embedding_dim = 100  # You can experiment with different embedding dimensions
num_epochs = 50
learning_rate = 0.001
batch_size = 32

dataset = TripleDataset(test_triples)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize ComplEx model
model = ComplEx(len(entities), len(relations), embedding_dim)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Step 3: Accuracy Calculation
def calculate_accuracy(model, triples):
    correct = 0
    total = len(triples)

    model.eval()
    with torch.no_grad():
        for head, relation, tail in triples:
            # Convert to tensors
            head = torch.tensor([head])
            relation = torch.tensor([relation])
            tail = torch.tensor([tail])

            # Get embeddings for the head, relation, and tail
            h_real = model.entity_embedding_real(head)
            h_imag = model.entity_embedding_imag(head)
            r_real = model.relation_embedding_real(relation)
            r_imag = model.relation_embedding_imag(relation)

            # Calculate the score for all entities as possible tails
            scores = []
            for t in range(len(entities)):
                t_real = model.entity_embedding_real(torch.tensor([t]))
                t_imag = model.entity_embedding_imag(torch.tensor([t]))
                score = (h_real * t_real + h_imag * t_imag) + \
                        (r_real * t_real + r_imag * t_imag) - \
                        (h_real * r_real + h_imag * r_imag)

                # Calculate the magnitude of the score (using L2 norm)
                norm_score = torch.norm(score, p=2).item()
                scores.append(norm_score)

            # Predict the tail with the highest score
            predicted_tail = np.argmax(scores)

            # Check if the predicted tail matches the true tail
            if predicted_tail == tail:
                correct += 1

    accuracy = correct / total
    return accuracy

# Step 4: Training loop with accuracy
for epoch in range(num_epochs):
    total_loss = 0
    model.train()
    for heads, relations, tails in dataloader:
        optimizer.zero_grad()

        # Forward pass
        predictions = model(heads, relations, tails)

        # Compute loss (mean squared error between predicted scores and true scores)
        targets = torch.zeros_like(predictions)  # Assuming correct triples have a score of 0
        loss = criterion(predictions, targets)

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Calculate accuracy after each epoch
    train_accuracy = calculate_accuracy(model, test_triples)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss:.4f}, Accuracy: {train_accuracy:.4f}")

print("Training complete.")


Epoch 1/50, Loss: 0.0248, Accuracy: 0.0000
Epoch 2/50, Loss: 0.0039, Accuracy: 0.0000
Epoch 3/50, Loss: 0.0015, Accuracy: 0.0000
Epoch 4/50, Loss: 0.0007, Accuracy: 0.0000
Epoch 5/50, Loss: 0.0004, Accuracy: 0.0000
Epoch 6/50, Loss: 0.0003, Accuracy: 0.0000
Epoch 7/50, Loss: 0.0002, Accuracy: 0.0000
Epoch 8/50, Loss: 0.0001, Accuracy: 0.0000
Epoch 9/50, Loss: 0.0001, Accuracy: 0.0000
Epoch 10/50, Loss: 0.0001, Accuracy: 0.0000
Epoch 11/50, Loss: 0.0001, Accuracy: 0.0000
Epoch 12/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 13/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 14/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 15/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 16/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 17/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 18/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 19/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 20/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 21/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 22/50, Loss: 0.0000, Accuracy: 0.0000
Epoch 23/50, Loss: 0.0000, Accuracy: 0.00